In [68]:
import cv2
import numpy as np
from numpy import cos, sin, deg2rad, roll, logical_and, where, inner, exp, rad2deg, arctan2, trace, dot, convolve, sqrt, subtract, log, floor, stack, delete, concatenate, max
from numpy.linalg import det, lstsq, norm
from einops import rearrange
from sift import sift
from stero import matchKeyPoints
from random import shuffle

In [58]:
# Load the images
image1 = cv2.imread('images/test1.jpeg', cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread('images/test2.jpeg', cv2.IMREAD_GRAYSCALE)

# reduce image size
xscale = 0.5
yscale = 0.5
image1_small = cv2.resize(image1, (0,0), fx = xscale, fy = yscale)
image2_small = cv2.resize(image2, (0,0), fx = xscale, fy = yscale)

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

# keypoints1, descriptors1 = sift(image1_small)
# keypoints2, descriptors2 = sift(image2_small)



In [59]:
def matchKeyPoints(keypoints1, keypoints2, descriptors1, descriptors2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    return good_matches

In [64]:
good_matches = matchKeyPoints(keypoints1, keypoints2, descriptors1, descriptors2)

In [60]:
# Draw matches
# matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
# matched_image_small = cv2.resize(matched_image, (0,0), fx = 0.5, fy = 0.5)

# # Display the matched image
# cv2.imshow('Matches', matched_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [69]:
shuffle(good_matches)


In [74]:
len(good_matches)

1104

In [75]:

image1_points = np.zeros((8))
image2_points = np.zeros((8))


for i in range(4):
    image1_idx = good_matches[i].queryIdx
    image2_idx = good_matches[i].trainIdx
    x1, y1 = keypoints1[image1_idx].pt
    x2, y2 = keypoints2[image1_idx].pt
    image1_points[2*i] = x1
    image1_points[2*i+1] = y1
    image2_points[2*i] = x2
    image2_points[2*i+1] = y2



### now work on get transformation
# estimate the fundamental matrix from corrspondane
f_unrolled = np.zeros(8)
A = np.zeros((8,8))


# decompose fundamental matrix to get essential matrix


In [63]:
def randsac(matches, num_trials):
    best_dist = 10000000
    best_trans = np.zeros((3,3))
    for n in num_trials:
        matches.shuffle()
        selected_pairs = matches[0:4,:]     # minimum number of points needed to conduct stero rectification
        
        # transformation = getSteroTransformation(selected_pairs)
        # transformed_points = matches[0] @ transformation
        # mean_dist = getMeanDistance(transformed_points, matches[1])
        # if mean_dist < best_dist:
        #     best_dist = mean_dist
        #     best_trans = transformation
    return best_trans


< cv2.DMatch 0x7a2985245750>

In [51]:
# # Initialize the Brute-Force matcher with L2 norm
# bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
# # Match descriptors using KNN algorithm (k=2)
# matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# # Apply Lowe's ratio test
# good_matches = []
# for m, n in matches:
#     if m.distance < 0.75 * n.distance:
#         good_matches.append(m)

# # Draw matches
# matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
# # matched_image_small = cv2.resize(matched_image, (0,0), fx = 0.5, fy = 0.5)


# # Display the matched image
# cv2.imshow('Matches', matched_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()